### TextRank

* Pagerank 알고리즘을 활용한 것
* Pagerank : 하이퍼링크를 가지는 웹 문서에 상대적 중요도에 따라 가중치를 부여하는 방법

`TextRank는 키워드 추출 기능과 핵심 문장 추출 기능, 두가지를 제공한다`

### Step1. 문서 타입에 따른 문장 단위로 분리하기 
#### (텍스트 크롤링 > 문장 단위 분리 > 명사 추출)

* 텍스트 크롤링
    * **newspaper** 패키지를 이용하여 text를 크롤링한다

* 문장 단위 분리
    * Kkma 함수 중 `sentences()` 라는 함수를 이용하녀 문장을 분리

* 명사 추출(POS-Tagging)
    * `TF-IDF 모델` 을 만들기 위한 전처리 과정으로 문장을 분리 한 뒤 문장을 형태소 단위로 나눈 후 품사 태깅을 통해 명사들만 추출한다. KoNLPy 중 `twitter`를 이용하여 명사를 추출한다

### < SentenceTokenizer Class >  
[텍스트 크롤링, 문장 단위 분리, 명사 추출] 과정을 Sentencetokenizer 클래스로 만듬

* stopwords 는 불용어로써 문장 내에서 내용을 나타내는데 의미를 가지지 않는 단어들의 집합을 의미한다. stopwords는 사용 목적에 맞게 적절히 추가, 삭제를 해주면 된다.

* **url2sentences(url)** : url 주소를 받아 기사내용(article.txt)을 추출하여 Kkma.sentences()를 이용하여 문장단위로 나누어 준 후 sentences를 return 해준다.

* **text2sentences(text0** : text(str)를 입력받아 Kkma.sentences()를 이용하여 문장단위로 나누어 준 후 sentences를 return 해준다

* **get_nouns(sentences)** : sentences를 받아 Twitter.nouns()를 이용하여 명사를 추출한 뒤 nouns를 return 해준다

In [17]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Twitter()
        self.stopwords = []
    
    def url2sentences(self, url):
        article = Article(url, language = 'ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx - 1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                      if noun not in self.stopwords and len(noun) > 1]))
                
        return nouns

### Step2. TF - IDF 모델 생성 및 그래프 생성


**TF-IDF**
* 정보 검색과 텍스트 마이닝에 사용하는 단어의 `상대적인 가중치`
* 여러 문서로 이루어진 문서 군이 있을 때, 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치

**TF**
* 단어 빈도, 특정 단어가 문서 내에 얼만큼의 빈도로 등장하는지를 나타내는 척도

**IDF**
* 역문헌 빈도수, 문서 빈도의 역수로써 전체 문서 개수를 해당 단어가 포함된 문서의 개수로 나눈 것을 의미한다

**TF-IDF 식**
* `Scikit-learn`을 이용하여 모델링 수행

**그래프 생성**
* Sentence-Term Matrix를 이용하여 전치행렬을 구한뒤 서로 곱해줌
* `Correlation Matrix`를 통해 문장(or 단어)간의 가중치 그래프로 나타낼 수 있음

### < GraphMatrix Class >

[TF-IDF 모델, 그래프 생성] 과정을 `GraphMatrix` 클래스로 구현했다

* **build_sent_graph(sentence)** : 명사로 이루어진 문장을 입력받아 sklearn의 TfidfVectorizer.fit_transform을 이용하여 tfidf matrix를 만든 후 Sentence graph를 return 한다

* **build_words_graph(sentence)** : 명사로 이루어진 문장을 입력받아 sklearn의 CountVectorizer.fit_transform을 이용하여 matrix를 만든 후 word graph와 (idx:word) 형태의 dictionary를 return 한다

In [21]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat =  normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float),
                                axis=0)
        vocab =  self.cnt_vec.vocabulary
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word]:word for word in vocab}

### Step3. TextRank 알고리즘 적용

앞에서 생성된 문장(or 단어)의 가중치 그래프를 이용하여 TextRank 알고리즘을 적용한다. TextRank 알고리즘을 통해 Ranking 값이 높은 순으로 정렬한 뒤 요약할 문장(or 단어)의 개수 만큼 출력해준다

* get_ranks(graph) : 위의 식을 구현한 부분이며, {idx : rank값} 형태의 dictionary를 return한다
* spookyQubit/TextRank 참고
* damping factor : 어떤 마구잡이로 웹서핑을 하는 사람이 그 페이지에 만족을 못하고 다른 페이지로 가는 링크를 클릭할 확률

In [9]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id,id] = 0 # diagonal 부분을 0 으로 (diagonal matrix는 대각행렬)
            link_sum = np.sum(A[:,id]) # A[:,id] = A[:][id]
            if link_sum != 0:
                A[:,id] /= link_sum
            A[:,id] *= -d
            A[id, id] = 1
        
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A,B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

### Step4. TextRank Class 구현

`Step1 ~ Step3` 까지의 클래스를 조합하여 TextRank 클래스 구현
* summarize(sent_num = 3) : Default로 `3줄 요약` 가능하게 구현
* keywords(word_num = 10) : Default로 `10개의 키워드`를 출력하도록 구현

In [15]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize =  SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key = lambda k:
                                          self.sent_rank_idx[k], reverse = True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k:
                                          self.word_rank_idx[k], reverse=True)
    
    def summarize(self, sent_num=3):
        summary = []
        index = []
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
    
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        
        keywords = []
        index = []
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        
        return keywords